In [1]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline  
import matplotlib.pyplot as plt
import ipywidgets as widgets
from brokenaxes import brokenaxes
import pandas as pd
from tqdm import tqdm

### PD Analysis

In [2]:
# GUI event handling functions

def show_plots(chkpt, attr1, attr2, attr3):
    
    global menu1_val, menu2_val
    
    # did menu2 change
    if attr2 != menu2_val:
        menu2_val = attr2
        if attr2=='Coarse':
            menu1.options = ['NIH-Shortcut','NIH-Shortcut (Migration)','NIH-Spurious (Age)','NIH-Spurious (Gender)']
            if menu1_val != 'NIH-Spurious (View)':
                attr1 = menu1_val

            # set slider params
            slider.value = 1
            chkpt = slider.value
            slider.min = 1
            slider.step = 1
            slider.description = 'Checkpoint: '

        elif attr2=='Fine':
            attr1 = menu1_val
            menu1.options = ['NIH-Shortcut','NIH-Spurious (Age)','NIH-Spurious (Gender)','NIH-Spurious (View)']

            # set slider params
            slider.min = 0
            slider.value = 0
            chkpt = slider.value
            slider.description = 'Iteration: '

    # did menu1 change
    if attr1 != menu1_val:       
        menu1_val = attr1
        if attr2=='Fine':
            chkpt = 0
            slider.value = chkpt
            
    if attr1=='NIH-Shortcut' and attr2=='Coarse':
        y_limit = 700
        slider.max = 25
        if attr3=='PD':
            pkl_path = f'/xxx/home/xxx/xxxp/xxx/projects/shortcut_detection_and_mitigation/data/nih/pd_analysis/imgsize_128/nih_shortcut_iterations/train_2k_ep{chkpt}.pkl'
        else:
            pkl_path = f'/xxx/home/xxx/xxxp/xxx/projects/shortcut_detection_and_mitigation/data/nih/layer_embeddings/imgsize_128/nih_shortcut_iterations/layer_ranks/ep{chkpt}.pkl'
    elif attr1=='NIH-Shortcut (Migration)':
        y_limit = 700
        slider.max = 25
        pkl_path = f'/xxx/home/xxx/xxxp/xxx/projects/shortcut_detection_and_mitigation/data/nih/pd_analysis/imgsize_128/nih_shortcut_iterations/train_2k_ep{chkpt}.pkl'
    elif attr1=='NIH-Spurious (Age)' and attr2=='Coarse':
        y_limit = 500
        slider.max = 10
        if chkpt>slider.max:
            chkpt = slider.max
            slider.value = chkpt
        if attr3=='PD':
            pkl_path = f'/xxx/home/xxx/xxxp/xxx/projects/shortcut_detection_and_mitigation/data/nih/pd_analysis/imgsize_128/nih_spurious_iterations/age_1p0_train_2k_ep{chkpt}.pkl'
        else:
            pkl_path = f'/xxx/home/xxx/xxxp/xxx/projects/shortcut_detection_and_mitigation/data/nih/layer_embeddings/imgsize_128/nih_spurious_iterations/layer_ranks/age_1p0_ep{chkpt}.pkl'
    elif attr1=='NIH-Spurious (Gender)' and attr2=='Coarse':
        y_limit = 700
        slider.max = 10
        if chkpt>slider.max:
            chkpt = slider.max
            slider.value = chkpt
        if attr3=='PD':
            pkl_path = f'/xxx/home/xxx/xxxp/xxx/projects/shortcut_detection_and_mitigation/data/nih/pd_analysis/imgsize_128/nih_spurious_iterations/gender_1p0_train_2k_ep{chkpt}.pkl'
        else:
            pkl_path = f'/xxx/home/xxx/xxxp/xxx/projects/shortcut_detection_and_mitigation/data/nih/layer_embeddings/imgsize_128/nih_spurious_iterations/layer_ranks/gender_1p0_ep{chkpt}.pkl'
    elif attr1=='NIH-Shortcut' and attr2=='Fine':
        y_limit = 150 # 1750
        slider.max = 5125
        slider.step = 205
        if attr3=='PD':
            pkl_path = f'/xxx/home/xxx/xxxp/xxx/projects/shortcut_detection_and_mitigation/data/nih/pd_analysis/imgsize_128/nih_shortcut_iterations/train_2k_ep1-it{chkpt}.pkl'        
        else:
            pkl_path = f'/xxx/home/xxx/xxxp/xxx/projects/shortcut_detection_and_mitigation/data/nih/layer_embeddings/imgsize_128/nih_shortcut_iterations/layer_ranks/ep1-it{chkpt}.pkl'
    elif attr1=='NIH-Spurious (Age)' and attr2=='Fine':
        y_limit = 150 # 1750   
        slider.max = 2520
        slider.step = 105
        if attr3=='PD':
            pkl_path = f'/xxx/home/xxx/xxxp/xxx/projects/shortcut_detection_and_mitigation/data/nih/pd_analysis/imgsize_128/nih_spurious_iterations/age_1p0_train_2k_ep1-it{chkpt}.pkl'
        else:
            pkl_path = f'/xxx/home/xxx/xxxp/xxx/projects/shortcut_detection_and_mitigation/data/nih/layer_embeddings/imgsize_128/nih_spurious_iterations/layer_ranks/age_1p0_ep1-it{chkpt}.pkl'
    elif attr1=='NIH-Spurious (Gender)' and attr2=='Fine':
        y_limit = 1000  
        slider.max = 2875
        slider.step = 115
        if attr3=='PD':
            pkl_path = f'/xxx/home/xxx/xxxp/xxx/projects/shortcut_detection_and_mitigation/data/nih/pd_analysis/imgsize_128/nih_spurious_iterations/gender_1p0_train_2k_ep1-it{chkpt}.pkl'
        else:
            pkl_path = f'/xxx/home/xxx/xxxp/xxx/projects/shortcut_detection_and_mitigation/data/nih/layer_embeddings/imgsize_128/nih_spurious_iterations/layer_ranks/gender_1p0_ep1-it{chkpt}.pkl'
    elif attr1=='NIH-Spurious (View)' and attr2=='Fine':
        y_limit = 750
        slider.max = 3000
        slider.step = 120
        if attr3=='PD':
            pkl_path = f'/xxx/home/xxx/xxxp/xxx/projects/shortcut_detection_and_mitigation/data/nih/pd_analysis/imgsize_128/nih_spurious_iterations/view_1p0_train_2k_ep1-it{chkpt}.pkl'
        else:
            pkl_path = f'/xxx/home/xxx/xxxp/xxx/projects/shortcut_detection_and_mitigation/data/nih/layer_embeddings/imgsize_128/nih_spurious_iterations/layer_ranks/view_1p0_ep1-it{chkpt}.pkl'            
    
    
    # plot PDs histogram
    with open(pkl_path, 'rb') as handle:
        batch_info = pickle.load(handle)

    if attr1=='NIH-Shortcut (Migration)':
        plt.figure(figsize=(14,7))
        for plt_idx in range(2):
            pkl_path1 = f'/xxx/home/xxx/xxxp/xxx/projects/shortcut_detection_and_mitigation/data/nih/pd_analysis/imgsize_128/nih_shortcut_iterations/train_2k_ep1.pkl'
            with open(pkl_path1, 'rb') as handle1:
                b1 = pickle.load(handle1)
                if plt_idx==0:
                    b2 = batch_info
                else:
                    b2 = b1
            b1['paths'] = np.array(b1['paths'])
            b1['pd'] = np.array(b1['pd'])
            b2['paths'] = np.array(b2['paths'])
            b2['pd'] = np.array(b2['pd'])

            # 4 regions in ep1
            ep1a = b1['paths'][(b1['pd']>=0)&(b1['pd']<40)] # color c1
            ep1b = b1['paths'][(b1['pd']>=40)&(b1['pd']<80)] # color c2
            ep1c = b1['paths'][(b1['pd']>=80)&(b1['pd']<120)] # color c3
            ep1d = b1['paths'][b1['pd']<0] # color red

            # 4 regions in ep2
            ep2a = b2['paths'][(b2['pd']>=0)&(b2['pd']<40)] 
            ep2b = b2['paths'][(b2['pd']>=40)&(b2['pd']<80)] 
            ep2c = b2['paths'][(b2['pd']>=80)&(b2['pd']<120)] 
            ep2d = b2['paths'][b2['pd']<0] 

            c1 = np.array([0,0,0,0]) # how ep1a is spread out in ep2
            c2 = np.array([0,0,0,0]) # how ep1b is spread out in ep2
            c3 = np.array([0,0,0,0]) # how ep1c is spread out in ep2
            c4 = np.array([0,0,0,0]) # how ep1d is spread out in ep2
            color1 = 'g'
            color2 = 'y'
            color3 = 'b'
            color4 = 'r'

            for idx,group in enumerate([ep2a,ep2b,ep2c,ep2d]):
                for path in group:
                    if path in ep1a:
                        c1[idx] = c1[idx] + 1
                    elif path in ep1b:
                        c2[idx] = c2[idx] + 1
                    elif path in ep1c:
                        c3[idx] = c3[idx] + 1
                    elif path in ep1d:
                        c4[idx] = c4[idx] + 1

            N = 4
            ind = np.arange(N)  
            width = 0.35 
            
            plt.subplot(1,2,2-plt_idx)
#             p1 = plt.bar(ind, c1, width, color='g')
#             p2 = plt.bar(ind, c2, width, bottom = c1, color='y')
#             p3 = plt.bar(ind, c3, width, bottom = c1+c2, color='b')
#             p4 = plt.bar(ind, c4, width, bottom = c1+c2+c3, color='r')
            p1 = plt.bar(ind, c1, width, color='b')
            p2 = plt.bar(ind, c2, width, bottom = c1, color='b')
            p3 = plt.bar(ind, c3, width, bottom = c1+c2, color='b')
            p4 = plt.bar(ind, c4, width, bottom = c1+c2+c3, color='r')

            plt.xticks(ind, ('R1 [PD=0-40]', 'R2 [PD=40-80]', 'R3 [PD=80-120]', 'Undefined'))
            if plt_idx==1:
                plt.title(f'Reference Plot (Epoch-1)')
            else:
                plt.title(f'Epoch-{chkpt}')
            plt.ylabel('# of images')
            plt.xlabel('Regions')     
            plt.ylim((0,1050))   
        plt.show()
    elif attr3=='PD':
        batch_info['pd'] = np.array(batch_info['pd'])
        batch_info['labels'] = np.array(batch_info['labels'])
        batch_info['preds'] = np.array(batch_info['preds'])

        pred_cls = (batch_info['preds']>0.5).astype(int)
        correct_preds_arr = (pred_cls==batch_info['labels'])
        pos_pd_arr = (batch_info['pd']>=0)

        plt.figure(figsize=(7,7))      
        baxes = brokenaxes(xlims=((0,120),(1e+5,100010)), hspace=.05)
        baxes.set_title(f'Prediction Depths ({slider.description}{chkpt})')
        baxes.set_ylabel('# of Images')
        baxes.set_xlabel('Layer')  
        baxes.set_ylim((0,y_limit))
        baxes.hist(batch_info['pd'][correct_preds_arr & pos_pd_arr],bins=30,color='b',alpha=1)
#         baxes.hist(batch_info['pd'][~correct_preds_arr & pos_pd_arr],bins=30,color='r',alpha=0.55)
        batch_info['pd'][~pos_pd_arr] = 100005
        baxes.hist(batch_info['pd'][~pos_pd_arr],bins=1,color='r',alpha=1)
        
#         x2 = range(4,120,4)
#         y2 = [98304,163840,229376,40960,57344,73728,90112,106496,122880,18432,22528,26624,30720,34816,38912,43008,47104,51200,55296,59392,63488,8704,9728,10752,11776,12800,13824,14848,15872]
#         plt.figure(figsize=(7,3))
#         plt.title(f'Densenet-121 (intermediate features)')
#         plt.ylabel('Dimensionality')
#         plt.xlabel('Layer')
#         plt.plot(x2,y2)
    else:
        x = range(4,120,4)
        plt.title('Layer Rank')
        plt.xlabel('Layer #')
        plt.ylabel('Rank')
        plt.plot(x,batch_info)
        # plt.ylim((0,950))
        # _ = plt.yticks([0,100,200,300,400,500,600,700,800,900,1000])


In [3]:
# GUI widgets

menu1 = widgets.Dropdown(
    options=['NIH-Shortcut','NIH-Shortcut (Migration)','NIH-Spurious (Age)','NIH-Spurious (Gender)'],
    value='NIH-Shortcut',
    description='Experiment: ',
    disabled=False,
)

menu2 = widgets.Dropdown(
    options=['Coarse','Fine'],
    value='Coarse',
    description='Type: ',
    disabled=False,
)

menu3 = widgets.Dropdown(
    options=['PD','Rank'],
    value='PD',
    description='Type: ',
    disabled=False,
)


slider = widgets.IntSlider(value=1,
                           min=1,
                           max=25,
                           step=1,
                           description='Checkpoint: ',
                           disabled=False,
                           continuous_update=False,
                           orientation='horizontal',
                           readout=True,
                           readout_format='d',
                           msg_throttle=1)

# global vars
menu1_val = menu1.value
menu2_val = menu2.value

In [5]:
# interactive GUI

_ = widgets.interact(show_plots, chkpt=slider, attr1=menu1, attr2=menu2, attr3=menu3)

interactive(children=(IntSlider(value=1, continuous_update=False, description='Checkpoint: ', max=25, min=1), …

In [5]:
pkl_path1 = f'/xxx/home/xxx/xxxp/xxx/projects/shortcut_detection_and_mitigation/data/nih/pd_analysis/imgsize_128/nih_shortcut_iterations/train_2k_ep1.pkl'
pkl_path2 = f'/xxx/home/xxx/xxxp/xxx/projects/shortcut_detection_and_mitigation/data/nih/pd_analysis/imgsize_128/nih_shortcut_iterations/train_2k_ep25.pkl'
with open(pkl_path1, 'rb') as handle1:
    b1 = pickle.load(handle1)
with open(pkl_path2, 'rb') as handle2:
    b2 = pickle.load(handle2)
b1['paths'] = np.array(b1['paths'])
b1['pd'] = np.array(b1['pd'])
b2['paths'] = np.array(b2['paths'])
b2['pd'] = np.array(b2['pd'])

In [ ]:
# 4 regions in ep1
ep1a = b1['paths'][(b1['pd']>=0)&(b1['pd']<40)] # color c1
ep1b = b1['paths'][(b1['pd']>=40)&(b1['pd']<80)] # color c2
ep1c = b1['paths'][(b1['pd']>=80)&(b1['pd']<120)] # color c3
ep1d = b1['paths'][b1['pd']<0] # color red

In [ ]:
# 4 regions in ep2
ep2a = b2['paths'][(b2['pd']>=0)&(b2['pd']<40)] 
ep2b = b2['paths'][(b2['pd']>=40)&(b2['pd']<80)] 
ep2c = b2['paths'][(b2['pd']>=80)&(b2['pd']<120)] 
ep2d = b2['paths'][b2['pd']<0] 

In [ ]:
c1 = np.array([0,0,0,0]) # how ep1a is spread out in ep2
c2 = np.array([0,0,0,0]) # how ep1b is spread out in ep2
c3 = np.array([0,0,0,0]) # how ep1c is spread out in ep2
c4 = np.array([0,0,0,0]) # how ep1d is spread out in ep2

count1 = np.array([0,0,0,0]) # count of correct predictions in ep1
count2 = np.array([0,0,0,0]) 
count3 = np.array([0,0,0,0]) 
count4 = np.array([0,0,0,0]) 

count5 = np.array([0,0,0,0]) # count of correct predictions in ep2
count6 = np.array([0,0,0,0]) 
count7 = np.array([0,0,0,0]) 
count8 = np.array([0,0,0,0]) 

color1 = 'g'
color2 = 'y'
color3 = 'b'
color4 = 'r'

for idx,group in enumerate([ep2a,ep2b,ep2c,ep2d]):
    for path in group:

        # check if this img was predicted correctly in the first epoch
        img_idx1 = b1['paths'].tolist().index(path)
        img_pred1 = int((b1['preds'][img_idx1]>0.5))
        img_lbl1 = b1['labels'][img_idx1]
        
        # check if this img was predicted correctly in the new epoch
        img_idx2 = b2['paths'].tolist().index(path)
        img_pred2 = int((b2['preds'][img_idx2]>0.5))
        img_lbl2 = b2['labels'][img_idx2]

        if path in ep1a:
            c1[idx] = c1[idx] + 1            
            if img_pred1==img_lbl1:
                count1[idx] = count1[idx] + 1
            if img_pred2==img_lbl2:
                count5[idx] = count5[idx] + 1
        elif path in ep1b:
            c2[idx] = c2[idx] + 1            
            if img_pred1==img_lbl1:
                count2[idx] = count2[idx] + 1
            if img_pred2==img_lbl2:
                count6[idx] = count6[idx] + 1
        elif path in ep1c:
            c3[idx] = c3[idx] + 1            
            if img_pred1==img_lbl1:
                count3[idx] = count3[idx] + 1
            if img_pred2==img_lbl2:
                count7[idx] = count7[idx] + 1
        elif path in ep1d:
            c4[idx] = c4[idx] + 1            
            if img_pred1==img_lbl1:
                count4[idx] = count4[idx] + 1
            if img_pred2==img_lbl2:
                count8[idx] = count8[idx] + 1

In [ ]:
c1,count1*100/c1,count5*100/c1

In [ ]:
c2,count2*100/c2,count6*100/c2

In [ ]:
c3,count3*100/c3,count7*100/c3

In [ ]:
(0.840*44 + 0.797*178 + 0.607*181)/(44+178+181), (0.794*34 + 0.783*111 + 0.650*223)/(34+111+223)

In [ ]:
N = 4
ind = np.arange(N)  
width = 0.35 
 
fig = plt.subplots(figsize =(10, 7))
p1 = plt.bar(ind, c1, width, color='g')
p2 = plt.bar(ind, c2, width, bottom = c1, color='y')
p3 = plt.bar(ind, c3, width, bottom = c1+c2, color='b')
p4 = plt.bar(ind, c4, width, bottom = c1+c2+c3, color='r')

plt.xticks(ind, ('R1 [PD=0-40]', 'R2 [PD=40-80]', 'R3 [PD=80-120]', 'Undefined'))
plt.title('Migratory Patterns')
plt.ylabel('# of images')
plt.xlabel('Regions')
# plt.yticks(np.arange(0, 81, 10))
# plt.legend((p1[0], p2[0], p3[0]), ('boys', 'girls'))
 
plt.show()

In [ ]:
b2.keys()

In [ ]:
np.where(b1['paths']==b2['paths'][50])

In [ ]:
b1['paths'].tolist().index(b2['paths'][50])

In [ ]:

import plotly.express as px

df = px.data.stocks(indexed=True)
fig = px.line(df,line_shape='spline')
fig.add_hline(y=1, line_dash="dot",
              annotation_text="Jan 1, 2018 baseline", 
              annotation_position="bottom right",
              annotation_font_size=20,
              annotation_font_color="blue"
             )
fig.add_vrect(x0="2018-09-24", x1="2018-12-18", 
              annotation_text="decline", annotation_position="top left",
              annotation=dict(font_size=20, font_family="Times New Roman"),
              fillcolor="green", opacity=0.25, line_width=0)
fig.show()

In [ ]:
fig.add_trace(go.Scatter(
    x=df['timestamp'],
    y=df['values'],
    mode='lines',
    line_shape='spline',
))

In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode()

xs = np.linspace(0, 20, 100)
ys = np.sin(xs) + xs*0.1
df = pd.DataFrame({'x': xs, 'y': ys})
theshhold = 1.0

fig = go.Figure()
# Full line

fig.add_scattergl(x=xs, y=df.y, line={'color': 'black'})

# Above threshhgold
fig.add_scattergl(x=xs, y=df.y.where(df.y >= 1), line={'color': 'red'})



xs = np.linspace(0, 20, 100)
ys = np.cos(xs) + xs*0.5
df = pd.DataFrame({'x': xs, 'y': ys})
theshhold = 1.0

# fig = go.Figure()
# Full line
fig.add_scattergl(x=xs, y=df.y, line={'color': 'black'})

# Above threshhgold
fig.add_scattergl(x=xs, y=df.y.where(df.y >= 1), line={'color': 'red'})


iplot(fig)

In [ ]:
for i in range(25):
    pkl_path = f'/xxx/home/xxx/xxxp/xxx/projects/shortcut_detection_and_mitigation/data/nih/pd_analysis/imgsize_128/nih_shortcut_iterations/train_2k_ep{i+1}.pkl'
    with open(pkl_path, 'rb') as handle:
        ep_info = pickle.load(handle)

    ep_info['labels'] = np.array(ep_info['labels'])
    ep_info['preds'] = np.array(ep_info['preds'])
    pred_cls = (ep_info['preds']>0.5).astype(int)
    correct_preds_arr = (pred_cls==ep_info['labels'])

    if i==0:
        arr1 = np.array(ep_info['pd'])
        arr2 = correct_preds_arr
    else:
        arr1 = np.vstack((arr1,np.array(ep_info['pd'])))
        arr2 = np.vstack((arr2,correct_preds_arr))

In [ ]:
df1 = pd.DataFrame(arr1)
df2 = pd.DataFrame(arr2)
df2

In [ ]:
import numpy as np
import pandas as pd
import time
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
from IPython import display
import random
init_notebook_mode()


for k in range(2000):
    i = int(2000*random.random())
    fig = go.Figure(layout_yaxis_range=[-35,120])
    fig.update_yaxes(dtick=40)
    fig.add_hrect(y0=-40, y1=0, 
                annotation_text="Undefined", annotation_position="top left",
                annotation=dict(font_size=15, font_family="Times New Roman"),
                fillcolor="red", opacity=0.25, line_width=0)
    fig.add_hrect(y0=0, y1=40, 
                annotation_text="R1", annotation_position="top left",
                annotation=dict(font_size=15, font_family="Times New Roman"),
                fillcolor="lightgreen", opacity=0.25, line_width=0)
    fig.add_hrect(y0=40, y1=80, 
                annotation_text="R2", annotation_position="top left",
                annotation=dict(font_size=15, font_family="Times New Roman"),
                fillcolor="mediumseagreen", opacity=0.25, line_width=0)
    fig.add_hrect(y0=80, y1=120, 
                annotation_text="R3", annotation_position="top left",
                annotation=dict(font_size=15, font_family="Times New Roman"),
                fillcolor="seagreen", opacity=0.25, line_width=0)
    xs = np.linspace(1,25,25)
    theshhold = 1.0


    green_marks = dict(color='forestgreen',
                        size=15,
                        line=dict(color='lawngreen',
                                    width=2))
    fig.add_scatter(x=xs, y=df1[i], line={'color': 'gray'},mode='lines+markers',marker=green_marks,
                   name='Correctly Classified')


    # Color the wrongly classified samples in Red
    red_marks = dict(color='darkred',
                        size=15,
                        line=dict(color='red',
                                    width=2))
    fig.add_scatter(x=xs, y=df1[i].where(df2[i]==False), line={'color': 'gray'},mode='lines+markers',
                    marker=red_marks,name='Wrongly Classified')

    fig.update_layout(title=f'Particle Flow Trajectory {i}',title_x=0.45,xaxis_title='Epoch',yaxis_title='Prediction Depth')

    display.clear_output(wait=True)
    display.display(iplot(fig))
    
    time.sleep(5)


In [ ]:
import numpy as np
import pandas as pd
import time
import random
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
from IPython import display
init_notebook_mode()

fig = go.Figure(layout_yaxis_range=[-35,120])
fig.update_yaxes(dtick=40)
fig.add_hrect(y0=-40, y1=0, 
            annotation_text="Undefined", annotation_position="top left",
            annotation=dict(font_size=15, font_family="Times New Roman"),
            fillcolor="red", opacity=0.25, line_width=0)
fig.add_hrect(y0=0, y1=40, 
            annotation_text="R1", annotation_position="top left",
            annotation=dict(font_size=15, font_family="Times New Roman"),
            fillcolor="lightgreen", opacity=0.25, line_width=0)
fig.add_hrect(y0=40, y1=80, 
            annotation_text="R2", annotation_position="top left",
            annotation=dict(font_size=15, font_family="Times New Roman"),
            fillcolor="mediumseagreen", opacity=0.25, line_width=0)
fig.add_hrect(y0=80, y1=120, 
            annotation_text="R3", annotation_position="top left",
            annotation=dict(font_size=15, font_family="Times New Roman"),
            fillcolor="seagreen", opacity=0.25, line_width=0)
xs = np.linspace(1,25,25)
theshhold = 1.0

inds_r1 = [805,1185,231,1397,919]
inds_r2 = [1285,907,287,1663,447,856]
inds_r3 = [559,1887,984,3,1281,1218,704]
inds_noisy = [1233,85,1479,296,406,1961,1054,874,1005,707]
inds_inactive = [589,455,523,1033,31,848,106,80,378,283]
all_inds = inds_r1+inds_r2+inds_r3+inds_inactive+inds_noisy

for i in tqdm(all_inds):

    green_marks = dict(color='forestgreen',
                        size=15,
                        line=dict(color='lawngreen',
                                    width=2))
    fig.add_scatter(x=xs, y=df1[i], line={'color': 'gray'},mode='lines+markers',marker=green_marks)


    # Color the wrongly classified samples in Red
    red_marks = dict(color='darkred',
                        size=15,
                        line=dict(color='red',
                                    width=2))
    fig.add_scatter(x=xs, y=df1[i].where(df2[i]==False), line={'color': 'gray'},mode='lines+markers',
                    marker=red_marks)

fig.update_layout(title='Particle Flow Trajectories',title_x=0.45,xaxis_title='Epoch',yaxis_title='Prediction Depth',showlegend=False)

display.clear_output(wait=True)
display.display(iplot(fig))




In [ ]:
import numpy as np
import pandas as pd
import time
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
from IPython import display
import random
init_notebook_mode()

inds_r1 = [452,961,805,1185,975,1508,355,612,231,1397,919,1511]
inds_r2 = [1285,907,225,1376,552,287,1036,1663,447,703,856,1760,982,1246,370,385]
inds_r3 = [1157,559,704,1745,798,1666,633,1887,270,1867,984,3,1218,609,1281,1390,42,982,1556,23]
inds_noisy = [1233,85,1479,296,406,1961,1054,874,1005,707]
inds_inactive = [589,455,523,210,1033,31,830,848,106,80,378,283,839]

for i in inds_inactive:
    fig = go.Figure(layout_yaxis_range=[-35,120])
    fig.update_yaxes(dtick=40)
    fig.add_hrect(y0=-40, y1=0, 
                annotation_text="Undefined", annotation_position="top left",
                annotation=dict(font_size=15, font_family="Times New Roman"),
                fillcolor="red", opacity=0.25, line_width=0)
    fig.add_hrect(y0=0, y1=40, 
                annotation_text="R1", annotation_position="top left",
                annotation=dict(font_size=15, font_family="Times New Roman"),
                fillcolor="lightgreen", opacity=0.25, line_width=0)
    fig.add_hrect(y0=40, y1=80, 
                annotation_text="R2", annotation_position="top left",
                annotation=dict(font_size=15, font_family="Times New Roman"),
                fillcolor="mediumseagreen", opacity=0.25, line_width=0)
    fig.add_hrect(y0=80, y1=120, 
                annotation_text="R3", annotation_position="top left",
                annotation=dict(font_size=15, font_family="Times New Roman"),
                fillcolor="seagreen", opacity=0.25, line_width=0)
    xs = np.linspace(1,25,25)
    theshhold = 1.0


    green_marks = dict(color='forestgreen',
                        size=15,
                        line=dict(color='lawngreen',
                                    width=2))
    fig.add_scatter(x=xs, y=df1[i], line={'color': 'gray'},mode='lines+markers',marker=green_marks,
                   name='Correctly Classified')


    # Color the wrongly classified samples in Red
    red_marks = dict(color='darkred',
                        size=15,
                        line=dict(color='red',
                                    width=2))
    fig.add_scatter(x=xs, y=df1[i].where(df2[i]==False), line={'color': 'gray'},mode='lines+markers',
                    marker=red_marks,name='Wrongly Classified')

    fig.update_layout(title=f'Particle Flow Trajectory {i}',title_x=0.45,xaxis_title='Epoch',yaxis_title='Prediction Depth')

    display.clear_output(wait=True)
    display.display(iplot(fig))
    
    time.sleep(5)
